# Initial Cleaning/ setup procedures


#### _1. Import libraries and set path to pkl file_

In [5]:
import pandas as pd
import os
from datetime import date, timedelta
import datetime
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data','data', 'weathersmallraw.pkl')
pkl_loc

'C:\\Users\\paull\\OneDrive - Scandinavian Tobacco Group A S\\Documents\\GitHub\\Weatherman\\data\\data\\weathersmallraw.pkl'

### _2. Create dataframe and run basic eda_

In [6]:
df = pd.read_pickle(pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8143419 entries, ('0CNUO', Timestamp('2021-10-15 00:00:00')) to ('ZYITU', Timestamp('2022-10-15 23:00:00'))
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   temp    float64
 1   dwpt    float64
 2   rhum    float64
 3   prcp    float64
 4   snow    float64
 5   wdir    float64
 6   wspd    float64
 7   wpgt    float64
 8   pres    float64
 9   tsun    float64
 10  coco    float64
dtypes: float64(11)
memory usage: 714.8+ MB


In [7]:
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','octweatherraw.pkl')
pkl_loc
df2 = pd.read_pickle(pkl_loc)
df2.info()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\paull\\OneDrive - Scandinavian Tobacco Group A S\\Documents\\GitHub\\Weatherman\\data\\data\\octweatherraw.pkl'

In [10]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,8143364.000,8143349.000,8143369.000,8108127.000,0.000,8143263.000,8143299.000,0.000,8143108.000,0.000,3232619.000
mean,10.367,4.488,70.391,0.117,NaN,176.150,12.871,NaN,1016.337,NaN,2.976
std,12.527,11.790,19.645,0.920,NaN,112.485,9.181,NaN,7.606,NaN,2.736
min,-43.000,-52.900,0.000,0.000,NaN,0.000,0.000,NaN,880.000,NaN,1.000
25%,1.400,-3.400,57.000,0.000,NaN,80.000,7.000,NaN,1012.000,NaN,1.000
50%,11.700,5.100,73.000,0.000,NaN,190.000,11.200,NaN,1016.400,NaN,3.000
75%,20.000,14.400,86.000,0.000,NaN,270.000,18.400,NaN,1021.000,NaN,3.000
max,65.000,60.000,100.000,332.000,NaN,360.000,243.000,NaN,1061.000,NaN,25.000


In [9]:
df = pd.concat([df, df2])
df.shape


NameError: name 'df2' is not defined

### _3. Remove unneeded columns_


In [11]:
df.drop(columns=['snow', 'wpgt','tsun','coco'],inplace=True)
df.head()

temp  dwpt   rhum  prcp    wdir   wspd     pres
station time                                                                
0CNUO   2021-10-15 00:00:00 9.900 7.800 87.000 0.100 226.000 20.900 1002.800
        2021-10-15 01:00:00 9.800 7.700 87.000 0.100 227.000 20.200 1003.500
        2021-10-15 02:00:00 9.600 7.400 86.000 0.100 228.000 19.400 1004.000
        2021-10-15 03:00:00 9.400 7.000 85.000 0.100 231.000 18.400 1004.400
        2021-10-15 04:00:00 9.100 6.900 86.000 0.100 233.000 18.400 1004.800

In [12]:
df=df.reset_index()

### _4. Counting Stations_

In [13]:
df.groupby("station").count() #clearly some stations are missing

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
station,,,,,,,,
0CNUO,8776,8776,8776,8776,8774,8776,8776,8776
0CO7B,8776,8776,8776,8776,8774,8776,8776,8776
0FV1F,8776,8776,8776,8776,8774,8776,8776,8776
0NNEW,3888,3888,3888,3888,3872,3888,3888,3875
1CZCV,4210,4210,4210,4210,4173,4210,4210,4210
...,...,...,...,...,...,...,...,...
ZFZUV,8784,8784,8784,8784,8782,8784,8784,8784
ZNWZW,4210,4210,4210,4210,4190,4210,4210,4210
ZUQJS,3894,3894,3894,3894,3881,3894,3894,3883


In [14]:
dfstations=df.groupby("station").count()
df_stationskeep = dfstations[dfstations['time']>5000].reset_index() #creating this to know which stations to keep later

In [15]:
df.time.describe()

C:\Users\paull\AppData\Local\Temp\ipykernel_35476\3482922840.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.time.describe()


count                 8143419
unique                   8784
top       2022-07-20 15:00:00
freq                      960
first     2021-10-15 00:00:00
last      2022-10-15 23:00:00
Name: time, dtype: object

In [16]:
cln_pkl_loc = os.path.join(ROOT_DIR, 'data_cleaning','cleanweather.pkl')
df.to_pickle(cln_pkl_loc)